In [1]:
# pip install pandas torchvision mlflow lightning captum

In [2]:
import pandas as pd
import numpy as np
import torchvision
import torch
import random
import os 
import json

# from tqdm import tqdm
from pathlib import Path
# from mouse_facial_expressions.data.datasets import Task1Folds
from mouse_facial_expressions.data.make_datasets import get_treatment_video_dataframe
from torch.utils.data import DataLoader

In [3]:
import mlflow
import matplotlib.pyplot as plt 
import pandas as pd

client = mlflow.MlflowClient('../mlruns')


In [4]:
experiment = client.search_experiments(filter_string="name='baseline'")[0]
# parent_run = client.search_runs(
#     experiment_ids=experiment.experiment_id,
#     filter_string="tags.mlflow.source.name='file:///home/andre/repos/mouse-facial-expressions-2023'")[0]

runs = client.search_runs(experiment_ids=experiment.experiment_id, order_by=["start_time"])
for run in runs:
    parent = client.get_parent_run(run.info.run_id)
    if parent is None:
        continue
    
    artifact_folder = Path(run.info.artifact_uri.split('://')[-1])
    meta_info_file = artifact_folder / 'meta_info/train_meta.json'
    with open(meta_info_file, 'r') as fp:
        meta_info = json.load(fp)
    testable_videos_mask = pd.read_csv(artifact_folder / 'testable_videos_mask/testable_videos_mask.csv', index_col=0)
    testable_mice = (testable_videos_mask==1).all(axis=1)
    
    # model_uri = run.info.artifact_uri + "/model"
    model_uri = str(artifact_folder / 'model')
    model = mlflow.pytorch.load_model(model_uri)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

df = get_treatment_video_dataframe()
df

,mouse,date_of_birth,treatment,injection_time,notes,image,video,recording,camera,year,...,day,hour,minutes,seconds,animal,start,end,discard,Notes,video_time
0,m1,18 April 2022,mid,12:23,NaN,m1_rec0_acclimation/frame06570.png,m1_rec0_acclimation,0,Basler_acA1920-40um,2022,...,13,11,5,36,m1,3:45,-1,NaN,NaN,11:05
1,m1,18 April 2022,mid,12:23,NaN,m1_rec0_acclimation/frame01690.png,m1_rec0_acclimation,0,Basler_acA1920-40um,2022,...,13,11,5,36,m1,3:45,-1,NaN,NaN,11:05
2,m1,18 April 2022,mid,12:23,NaN,m1_rec0_acclimation/frame06320.png,m1_rec0_acclimation,0,Basler_acA1920-40um,2022,...,13,11,5,36,m1,3:45,-1,NaN,NaN,11:05
3,m1,18 April 2022,mid,12:23,NaN,m1_rec0_acclimation/frame06550.png,m1_rec0_acclimation,0,Basler_acA1920-40um,2022,...,13,11,5,36,m1,3:45,-1,NaN,NaN,11:05
4,m1,18 April 2022,mid,12:23,NaN,m1_rec0_acclimation/frame11320.png,m1_rec0_acclimation,0,Basler_acA1920-40um,2022,...,13,11,5,36,m1,3:45,-1,NaN,NaN,11:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64579,f33,21 December 22,mid,15:06,NaN,f33_rec0_acclimation/frame12640.png,f33_rec0_acclimation,0,Basler_acA1920-40um,2023,...,21,13,16,58,f33,NaN,NaN,NaN,NaN,13:16
64580,f33,21 December 22,mid,15:06,NaN,f33_rec0_acclimation/frame11400.png,f33_rec0_acclimation,0,Basler_acA1920-40um,2023,...,21,13,16,58,f33,NaN,NaN,NaN,NaN,13:16
64581,f33,21 December 22,mid,15:06,NaN,f33_rec0_acclimation/frame20260.png,f33_rec0_acclimation,0,Basler_acA1920-40um,2023,...,21,13,16,58,f33,NaN,NaN,NaN,NaN,13:16
64582,f33,21 December 22,mid,15:06,NaN,f33_rec0_acclimation/frame12580.png,f33_rec0_acclimation,0,Basler_acA1920-40um,2023,...,21,13,16,58,f33,NaN,NaN,NaN,NaN,13:16
